In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import timedelta, date
import pandas as pd
from io import StringIO
import time

In [ ]:
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(1983, 1, 1)
end_date = date(1983, 12, 31)
final_df = None
links_visited = []
for single_date in daterange(start_date, end_date):
    links_visited.clear()
    print(single_date)
    url = f"https://www.basketball-reference.com/boxscores/?month={single_date.month}&day={single_date.day}&year={single_date.year}"
    time.sleep(4)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    if response.status_code != 200:
        print('Response code: ', response.status_code)
        print('Response headers: ', response.headers)
        print('Response text: ', response.text)
        break
    links = soup.find_all('a')
    for link in links:
        game_link = link.get('href')
        if game_link not in links_visited and "boxscores" in game_link and "html" in game_link:
            print(game_link)
            links_visited.append(game_link)
            time.sleep(4)
            game_response = requests.get(f"https://www.basketball-reference.com{game_link}")
            game_soup = BeautifulSoup(game_response.text, 'html.parser')
            for table in game_soup.find_all('table'):
                table_id = table.get('id')
                if table_id is None:
                    continue
                
                if "basic" in table_id or "advanced" in table_id:
                    column_names = [th.get_text() for th in table.find_all("tr")[1].find_all("th")]
                    df = pd.read_html(StringIO(str(table)), header=0)[0]
                    df.columns = column_names
                    df = df[df['Starters'] != 'Starters']
                    df = df[df['Starters'] != 'Reserves']
                    df = df[df['Starters'] != 'Team Totals']
                    df['Year'] = single_date.year
                    df['Month'] = single_date.month
                    df['Day'] = single_date.day
                    if final_df is None:
                        final_df = df
                    else:
                        final_df = pd.concat([final_df, df], axis=0)

In [ ]:
print(final_df)

In [ ]:
final_df.columns

In [ ]:
final_df

In [ ]:
import threading
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import time
from datetime import date, timedelta

def scrape_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    final_df = None
    if response.status_code != 200:
        print('Response code: ', response.status_code)
        print('Response headers: ', response.headers)
        print('Response text: ', response.text)
        break
    links = soup.find_all('a')
    for link in links:
        game_link = link.get('href')
        if game_link not in links_visited and "boxscores" in game_link and "html" in game_link:
            print(game_link)
            links_visited.append(game_link)
            time.sleep(4)
            game_response = requests.get(f"https://www.basketball-reference.com{game_link}")
            game_soup = BeautifulSoup(game_response.text, 'html.parser')
            for table in game_soup.find_all('table'):
                table_id = table.get('id')
                if table_id is None:
                    continue

                if "basic" in table_id or "advanced" in table_id:
                    column_names = [th.get_text() for th in table.find_all("tr")[1].find_all("th")]
                    df = pd.read_html(StringIO(str(table)), header=0)[0]
                    df.columns = column_names
                    df = df[df['Starters'] != 'Starters']
                    df = df[df['Starters'] != 'Reserves']
                    df = df[df['Starters'] != 'Team Totals']
                    df['Year'] = single_date.year
                    df['Month'] = single_date.month
                    df['Day'] = single_date.day
                    if final_df is None:
                        final_df = df
                    else:
                        final_df = pd.concat([final_df, df], axis=0)

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

def scrape_for_date(single_date):
    links_visited.clear()
    print(single_date)
    url = f"https://www.basketball-reference.com/boxscores/?month={single_date.month}&day={single_date.day}&year={single_date.year}"
    time.sleep(4)
    scrape_data(url)

if __name__ == "__main__":
    start_date = date(1983, 1, 1)
    end_date = date(1983, 12, 31)
    links_visited = []  # Initialize links_visited here

    threads = []
    for single_date in daterange(start_date, end_date):
        thread = threading.Thread(target=scrape_for_date, args=(single_date,))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    # Combine data from all threads into a final dataframe
    # You can create a global dataframe or append to a list and concatenate later
    # Example: final_df = pd.concat(list_of_dataframes)

    # Rest of your code
